# Plots for the "Illustrative" Section

In [ ]:
# using Yao, QAOA, Random, Combinatorics, Arpack, Distributions, LinearAlgebra, , FFTW
using QAOA, Distributions, Interpolations, LinearAlgebra
using HDF5, Printf
using PythonPlot
# PythonPlot.matplotlib.style.use("./paper.mplstyle")
PythonPlot.rc("axes", prop_cycle = PythonPlot.matplotlib.cycler(color = ["#2D5FAA", "#B7293F", "#438E6A", "#F7BD2E", "#F16C37"]))

PATH = "../";
# PATH = "/home/ubuntu/Archives/"
# PLOT_PATH = "/home/ubuntu/Archives/plots/SK_model/paper/";

In [ ]:
using Revise, SpinFluctuations

In [ ]:
Base.show(io::IO, f::Float64) = @printf(io, "%1.4f", f)

In [ ]:
N = 9
# N = 13
# N = 19

patterns_dict = Dict(
	9  => r"random_SK_instance_N_9_seed_(\d+)\.h5",
	11 => r"random_SK_instance_N_11_seed_(\d+)\.h5",
	13 => r"random_SK_instance_N_13_seed_(\d+)\.h5",
	15 => r"random_SK_instance_N_15_seed_(\d+)\.h5",
	17 => r"random_SK_instance_N_17_seed_(\d+)\.h5",
	19 => r"random_SK_instance_N_19_seed_(\d+)\.h5",
);

# folder_name = PATH * @sprintf("data/")
subdir = "small_gaps"
# subdir = "large_gaps"
folder_name = PATH * @sprintf("data/SK_model/N_%i/%s/", N, subdir)
instance_names = readdir(folder_name);

In [ ]:
# N = 9
# small 
# seed = 12778 # => 0.6085
# seed = 73014 # => 0.7356
# seed = 65343 # => 0.7854
# seed = 57220 # => 0.8052
# seed = 47297 # => 0.8217
# seed = 78653 # => 0.8303
# seed = 46944 # => 0.8403
# seed = 70592 # => 0.8467
# seed = 62565 # => 0.8595
# seed = 87929 # => 0.8867
seed = 39879 # => 1.0371, 5th excited state - hardest instance as example!
seed = 63685 # => 1.2428, 4th excited state

# seed = 38906
# seed = 23320

# seed = 11414 # not actually hard
# seed = 11048 # interesting

# looking at max. peaks
seed = 22824
# seed = 28563
# seed = 72723
# seed = 11456
# seed = 83260
# seed = 23583 # very high fraction peak, great illustration!
# seed = 78065

# seed = 50008 # lowest peak

# large
# seed = 100210

In [ ]:
# N = 13
# seed = 36733

# N = 19
# small
# seed = 538 # => 0.4583
# seed = 628 # => 0.4771
# seed = 147 # => 0.5004
# seed = 63 # => 0.5323
# seed = 914 # => 0.5950
# seed = 863 # => 0.7336
# seed = 699 # => 0.7725
# seed = 1238 # => 0.7753
# seed = 18 # => 0.8269
# seed = 319 # => 0.8990 - GREAT ILLUSTRATION!
# seed = 674 # => 0.9061
# seed = 554 # => 1.0416
# seed = 432
# seed = 1205

# not quite converged
# seed = 109
# seed = 117
# seed = 900 # very interesting

In [ ]:
h5open(folder_name * @sprintf("random_SK_instance_N_%i_seed_%i.h5", N, seed))

In [ ]:
# h5open(folder_name * @sprintf("results_random_SK_instance_N_%i_seed_%i.h5", N, seed))

In [ ]:
J_mat = h5read(folder_name * @sprintf("random_SK_instance_N_%i_seed_%i.h5", N, seed), "J")
mf_problem = Problem(0, J_mat);

λ = h5read(folder_name * @sprintf("random_SK_instance_N_%i_seed_%i.h5", N, seed), "exact_ARPACK_LM_eigvals");

In [ ]:
npts = 2048
# npts = 8192
coarse_times = range(0, 1, npts + 1);

## Exact gap

In [ ]:
# T_final = 32768
T_final = 32000
tol = 1e-8;

In [ ]:
exact_times = range(0, 1, 33);

In [ ]:
gap = λ[2, :] .- λ[1, :];
mingap = minimum(gap)
mingap |> println
gap_idx = findfirst(x -> x == mingap, gap)
gaploc = exact_times[gap_idx]
gaploc |> println

In [ ]:
small_idxs = findall(x -> x < 0.05, gap ./ (2pi))
gap_interval = exact_times[small_idxs];

In [ ]:
bogo_spec = h5read(folder_name * @sprintf("results_random_SK_instance_N_%i_seed_%i.h5", N, seed), @sprintf("bogoliubov_spectrum_T_final_%.0f_tol_1e%.0f_npts_%i", T_final, log10(tol), 32));

### Eigenstates

In [ ]:
all_eigvecs = h5read(folder_name * @sprintf("random_SK_instance_N_%i_seed_%i.h5", N, seed), "exact_ARPACK_LM_lowest_eigvecs");

In [ ]:
H_x = SpinFluctuations.hamiltonian(1, 0, mf_problem.local_fields, mf_problem.couplings)
H_z = SpinFluctuations.hamiltonian(0, 1, mf_problem.local_fields, mf_problem.couplings);

In [ ]:
gs = [all_eigvecs[k, :, 1] for k in 1:length(exact_times)]
first_ex = [all_eigvecs[k, :, 2] for k in 1:length(exact_times)]
second_ex = [all_eigvecs[k, :, 3] for k in 1:length(exact_times)]

overlap_01_x = [first_ex[k]' * H_x * gs[k] for k in 1:length(exact_times)]
overlap_01_z = [first_ex[k]' * H_z * gs[k] for k in 1:length(exact_times)]

overlap_02_x = [second_ex[k]' * H_x * gs[k] for k in 1:length(exact_times)]
overlap_02_z = [second_ex[k]' * H_z * gs[k] for k in 1:length(exact_times)];

In [ ]:
figure(figsize = (4, 4))

ax = subplot(211)
for i in 1:size(bogo_spec)[1]
	plot(exact_times, (λ[i, :] .- λ[1, :]), "-k", lw = 1, alpha = 1)
	plot(exact_times[2:end], bogo_spec[i, :], "--k", lw = 2, alpha=0.5)
end
# axvline(gap_interval[1], c="C0")
# axvline(gap_interval[end], c="C0")
plot([], [], "-k", lw = 1, label = "Exact")
plot([], [], label = "Bogoliubov", "--k", lw = 2, alpha = 0.5)
xlim(0.0, 1.0)
ylim(-0.0, 2)
ax.set_xticklabels([])
ylabel("\$E_\\alpha - E_0\$")
legend(frameon = false)

ax = subplot(212)
# plot(exact_times, ((λ[2, :] .- λ[1, :])), "--C0")
# plot(exact_times, ((λ[3, :] .- λ[1, :])), "--C1", lw = 2)
plot(exact_times, abs.(overlap_02_z .- overlap_02_x), label = "\$\\alpha = 2\$", "-C1", lw=2)
plot(exact_times, abs.(overlap_01_z .- overlap_01_x), label = "\$\\alpha = 1\$", "-C0")
xlim(0.0, 1.0)
ylim(0, 1)
xlabel("\$s\$")
ylabel("\${|\\langle \\alpha|H_Z-H_X|0\\rangle|}\$")

legend(frameon = false)

tight_layout()
display(gcf())
PythonPlot.close();

## Mean-field trajectories

In [ ]:
T_final = 32768.0
tol = 1e-6;

In [ ]:
sol_t = h5read(folder_name * @sprintf("results_random_SK_instance_N_%i_seed_%i.h5", N, seed), @sprintf("mean_field_T_final_%.0f_tol_1e%.0f/times", T_final, log10(tol)))
sol_u = h5read(folder_name * @sprintf("results_random_SK_instance_N_%i_seed_%i.h5", N, seed), @sprintf("mean_field_T_final_%.0f_tol_1e%.0f/trajectories", T_final, log10(tol)));

In [ ]:
size(sol_u)

### Bloch coordinates, Magnetization & Complex Coordinates

In [ ]:
nx_vals = n_vals("x", sol_u)
ny_vals = n_vals("y", sol_u)
nz_vals = n_vals("z", sol_u);

In [ ]:
nx_coarse = n_coarse(nx_vals, sol_t, coarse_times)
ny_coarse = n_coarse(ny_vals, sol_t, coarse_times)
nz_coarse = n_coarse(nz_vals, sol_t, coarse_times);

In [ ]:
S_vals = [transpose(reduce(hcat, [nx_coarse[:, k], ny_coarse[:, k], nz_coarse[:, k]])) |> Matrix for k in 1:npts+1]
magnetizations = reduce(hcat, map(S -> magnetization(S, mf_problem.local_fields, mf_problem.couplings), S_vals));

In [ ]:
areas = Dict()
# dts = [(x[2] - x[1]) / T_final for x in zip(sol_t[1:end-1], sol_t[2:end])]
dts = [(x[2] - x[1]) / T_final for x in zip(coarse_times[1:end-1], coarse_times[2:end])]
for spin_idx in 1:N-1
	# areas[spin_idx] = sum(dts .* nz_vals[spin_idx, 2:end]) |> abs
	areas[spin_idx] = sum(dts .* magnetizations[spin_idx, 2:end]) |> abs
end

top_idxs = [k for (k, v) in sort(areas |> collect, by = x -> x[2])]
top_idx = top_idxs[1]
# top_idx = top_idxs[2]

In [ ]:
regular_trajectories = filter!(x -> x != top_idx, collect(1:N-1));

In [ ]:
figure(figsize = (4, 6))

ax = subplot(411)
for spin_nr in regular_trajectories
	plot(sol_t ./ sol_t[end], nx_vals[spin_nr, :], "-", c = "k")
end
plot(sol_t ./ sol_t[end], nx_vals[top_idx, :], "-", c = "C1", label = @sprintf("\$i=%i\$", top_idx))
xlim(0, 1)
ax.set_xticklabels([])
ylim(-0.1, 1.1)
ylabel("\$n_i^x(s)\$")
legend(frameon = false, ncol = 2)

ax = subplot(412)
for spin_nr in regular_trajectories
	plot(sol_t ./ T_final, nz_vals[spin_nr, :], "-", c = "k")
end
plot(sol_t ./ T_final, nz_vals[top_idx, :], "-", c = "C1", label = @sprintf("\$i=%i\$", top_idx))
xlim(0, 1)
ylim(-1.1, 1.1)
ax.set_xticklabels([])
ylabel("\$n_i^z(s)\$")

ax = subplot(413)
for spin_nr in regular_trajectories
	plot(coarse_times, (abs.(complex_coordinate(spin_nr, nx_coarse, ny_coarse, nz_coarse)) .^ 2), "-", c = "k")
end
plot(coarse_times, (abs.(complex_coordinate(top_idx, nx_coarse, ny_coarse, nz_coarse)) .^ 2), "-", c = "C1", label = @sprintf("\$i=%i\$", top_idx))

xlim(0, 1)
ylim(0, 1.2)
ax.set_xticklabels([])
# ylabel("\$1 + [\\mathrm{Re}\\; z_i(s)]^2\$")
ylabel("\$|z_i(s)|^2\$")

ax = subplot(414)
inner_navg = 64
outer_navg = 64
for spin_nr in regular_trajectories
	# plot(coarse_times, smoothen(smoothen(magnetizations[spin_nr, :], coarse_times, navg=inner_navg), coarse_times, navg=outer_navg), "-k")
	plot(coarse_times, magnetizations[spin_nr, :], "-k")
end
# plot(coarse_times, smoothen(smoothen(magnetizations[top_idx, :], coarse_times, navg=inner_navg), coarse_times, navg=outer_navg), "-C1", label = @sprintf("\$i=%i\$", top_idx))
plot(coarse_times, magnetizations[top_idx, :], "-C1", label = @sprintf("\$i=%i\$", top_idx))
xlim(0.0, 1)
ylim(-3, 3)
xlabel("\$s\$")
ylabel("\$m_i(s)\$")

tight_layout()
display(gcf())
# savefig("../plots/" * @sprintf("mean_field_max2sat_typical_instance_%04i_from_arxiv_2206_06876_N_%i_num_clauses_%i.pdf", idx, N, num_clauses), dpi=256, bbox_inches="tight")
PythonPlot.close();

### Energies

In [ ]:
# sigma_star = sign.(sol.u[end][3, :])
sigma_star = sign.(sol_u[end, 3, :])
h = mf_problem.local_fields
J = mf_problem.couplings
E = sum([-h[l] * sigma_star[l] for l in 1:N-1]) + sum([-J[i, j] * sigma_star[i] * sigma_star[j] for i in 1:N-1 for j in (i+1):N-1])

In [ ]:
[λ[k, :][end] for k in 1:6] |> println

In [ ]:
isapprox.([λ[k, :][end] for k in 1:6], E, atol=1e-5) |> println

In [ ]:
for spin_idx in 1:N-1
	# sigma_star_flip = sign.(sol.u[end][3, :])
	sigma_star_flip = sign.(sol_u[end, 3, :])
	sigma_star_flip[spin_idx] = (-1) * sigma_star_flip[spin_idx]
	E_flip = sum([-h[l] * sigma_star_flip[l] for l in 1:N-1]) + sum([-J[i, j] * sigma_star_flip[i] * sigma_star_flip[j] for i in 1:N-1 for j in (i+1):N-1])
	println(spin_idx, ": ", E_flip)
end

In [ ]:
println(top_idxs)

## Statistical Green function

In [ ]:
tol = 1e-8
T_final = 32000.0;
# T_final = 32768.;
# T_final = 65536.0;

In [ ]:
# nx_coarse = reduce(hcat, [map(linear_interpolation(sol_t, nx_vals[spin_nr, :], extrapolation_bc=Line()), T_final .* coarse_times) for spin_nr in 1:N-1]) |> transpose
# nz_coarse = reduce(hcat, [map(linear_interpolation(sol_t, nz_vals[spin_nr, :], extrapolation_bc=Line()), T_final .* coarse_times) for spin_nr in 1:N-1]) |> transpose;

In [ ]:
all_flucs = h5read(folder_name * @sprintf("results_random_SK_instance_N_%i_seed_%i.h5", N, seed), @sprintf("fluctuations_T_final_%.0f_tol_1e%.0f_npts_%i", T_final, log10(1e-8), npts));

In [ ]:
regular_trajectories = filter!(x -> x != top_idxs[1], collect(1:N-1));
# regular_trajectories = filter!(x -> x != top_idxs[2], regular_trajectories);

scale_factors = [1 .+ abs.(complex_coordinate(i, nx_coarse, ny_coarse, nz_coarse)) .^ 2 for i in 1:N-1]
mean_flucs = mean([smoothen(all_flucs[i, :], coarse_times) for i in regular_trajectories], dims=1)[1];
mean_scaled_flucs = mean([smoothen(scale_factors[i] .^ 2 .* all_flucs[i, :], coarse_times) for i in regular_trajectories], dims = 1)[1];

In [ ]:
inner_navg = 64
outer_navg = 64 # 256

inv_mag = 1 ./ smoothen(smoothen(magnetizations[top_idx, :], coarse_times, navg=inner_navg) .|> abs, coarse_times, navg=outer_navg)
# inv_mag = 1 ./ (magnetizations[top_idx, :] .|> abs)
scale_factor = 1 .+ abs.(complex_coordinate(top_idx, nx_coarse, ny_coarse, nz_coarse)) .^ 2
crit_flucs = smoothen(scale_factor.^2 .* all_flucs[top_idx, :], coarse_times, navg=128) .* inv_mag;

In [ ]:
figure(figsize = (4, 4))
ax = subplot(211)
axvline(gaploc, ls = "-", c="k", alpha=0.2, lw=5)
semilogy(exact_times, abs.(overlap_01_z .- overlap_01_x) ./ ((λ[2, :] .- λ[1, :])), label = "\$\\alpha = 1\$")
semilogy(exact_times, abs.(overlap_02_z .- overlap_02_x) ./ ((λ[3, :] .- λ[1, :])), label="\$\\alpha = 2\$", lw=2)
xlim(0.0, 1.0)
ylim(1e-1, 1e3)
# ylim(0, 10)
ax.set_xticklabels([])
ylabel("\$\\frac{|\\langle \\alpha|H_Z-H_X|0\\rangle|}{E_\\alpha - E_0}\$")

legend(frameon = false)

subplot(212)

axvline(gaploc, ls = "-", c="k", alpha=0.2, lw=5)
plot(coarse_times, crit_flucs)

xlim(0, 1)
# ylim(1e-1, 1e2)
ylim(0, )
xlabel("\$s\$")
ylabel("\$ \\langle\\delta \\bar z_i(s) \\delta z_i(s) \\rangle / |m_i(s)|\$")

tight_layout()
display(gcf())
PythonPlot.close();

In [ ]:
figure(figsize = (8, 3))
ylims = (0, 0.2)
# ylims = (0, 0.3)

for i in 1:(N-1)÷2
	ax = subplot(2, (N - 1) ÷ 2, i)
	c = "k"
	if i in [top_idx]
		c = "C1"
	end
	scale_factor = 1 .+ abs.(complex_coordinate(i, nx_coarse, ny_coarse, nz_coarse)) .^ 2
	plot(coarse_times, scale_factor .^ 2 .* smoothen(all_flucs[i, :], coarse_times), label = @sprintf("\$i=%s\$", string(i)), c = c)
	xlim(0.0, 1.0)
	ax.set_xticklabels([])
	ylim(ylims...)
	if i > 1
		ax.set_yticklabels([])
	end
	legend(frameon = false, handlelength = 0)
end

ax = subplot(2, (N - 1) ÷ 2, 1)
ax.set_ylabel("\$ \\langle\\delta \\bar z_i(s) \\delta z_i(s) \\rangle\$")

for i in (N-1)÷2+1:N-1
	ax = subplot(2, (N - 1) ÷ 2, i)
	c = "k"
	if i in [top_idx]
		c = "C1"
	end
	scale_factor = 1 .+ abs.(complex_coordinate(i, nx_coarse, ny_coarse, nz_coarse)) .^ 2
	plot(coarse_times, scale_factor .^ 2 .* smoothen(all_flucs[i, :], coarse_times), label = @sprintf("\$i=%s\$", string(i)), c = c)
	xlim(0.0, 1.0)
	ylim(ylims...)
	if i > (N - 1) ÷ 2 + 1
		ax.set_yticklabels([])
	end
	xlabel("\$s\$")
	legend(frameon = false, handlelength = 0)
end

ax = subplot(2, (N - 1) ÷ 2, (N - 1) ÷ 2 + 1)
ax.set_ylabel("\$ \\langle\\delta \\bar z_i(s) \\delta z_i(s) \\rangle\$")

tight_layout()
display(gcf())
# savefig("../plots/" * @sprintf("fluctuations_max2sat_typical_instance_%04i_from_arxiv_2206_06876_N_%i_num_clauses_%i.pdf", idx, N, num_clauses), dpi=256, bbox_inches="tight")
PythonPlot.close();

## Spectrum

In [ ]:
T_final = 32768.0
tol = 1e-6
τ_final = 2048.0

npts_diag = 16
T_diags = T_final .* range(0.5, 1.0, npts_diag + 1)

ωs = []
spec_sum = []
for k in 1:17
	push!(ωs, h5read(folder_name * "results_" * @sprintf("random_SK_instance_N_%i_seed_%i.h5", N, seed), @sprintf("spectra_T_final_%i_tau_final_%i/T_%0.5f/omegas", T_final, τ_final, T_diags[k] / T_final)))
	push!(spec_sum, h5read(folder_name * "results_" * @sprintf("random_SK_instance_N_%i_seed_%i.h5", N, seed), @sprintf("spectra_T_final_%i_tau_final_%i/T_%0.5f/data", T_final, τ_final, T_diags[k] / T_final)))
end


In [ ]:
t_idx = 8
println(T_diags[t_idx] ./ T_final)

In [ ]:
figure(figsize = (3.2, 2.2))
ax = subplot(111)
axvline.(bogo_spec[N:end, 16+t_idx], c = "k", alpha = 0.5, ls = "--", lw=2)
for k in [t_idx]
	plot(2pi .* ωs[k], spec_sum[k])
end
xlim(0.4, 3.6)
ylim(-0.0, 1)
yticks([0, 0.5, 1.0])
xlabel("\$\\omega\$")
ylabel("\$\\mathrm{Tr}\\;{\\rho}(T, \\omega)\$", fontsize = 12)
tight_layout()
display(gcf())
PythonPlot.close();

## Old

In [ ]:
tol = 1e-8
T_final = 32768.0
npts = 4096
coarse_times_fine = range(0, 1, npts + 1);
all_flucs_fine = h5read(folder_name * @sprintf("results_random_SK_instance_N_%i_seed_%i.h5", N, seed), @sprintf("fluctuations_T_final_%.0f_tol_1e%.0f_npts_%i", T_final, log10(1e-8), npts));

In [ ]:
figure(figsize = (4, 3))
# plot(coarse_times, mean(new_all_flucs, dims=1)[1, :])
# plot(coarse_times, mean(all_flucs, dims=1)[1, :])
plot(coarse_times_fine, mean(all_flucs_fine, dims = 1)[1, :])
plot(coarse_times, smoothen(mean(all_flucs, dims = 1)[1, :], coarse_times), "-")
tight_layout()
xlim(0, 1)
ylim(0)
display(gcf())
tight_layout()
PythonPlot.close();


In [ ]:
figure(figsize = (4, 4))
ax = subplot(211)
# plot(exact_times, abs.(overlap_01_z .- overlap_01_x) ./ ((λ[2, :] .- λ[1, :])), label = "\$\\alpha = 1\$")
semilogy(exact_times, abs.(overlap_01_z .- overlap_01_x) ./ ((λ[2, :] .- λ[1, :])), label = "\$\\alpha = 1\$")
plot(exact_times, abs.(overlap_02_z .- overlap_02_x) ./ ((λ[3, :] .- λ[1, :])), label="\$\\alpha = 2\$", lw=2)
# semilogy(exact_times, (abs.(overlap_01_z .- overlap_01_x) ./ ((λ[2, :] .- λ[1, :]))) .+ (abs.(overlap_02_z .- overlap_02_x) ./ ((λ[3, :] .- λ[1, :]))), "-k", lw=2)
xlim(0.0, 1.0)
ylim(1e-1, 1e3)
# ylim(0, 10)
ax.set_xticklabels([])
ylabel("\$\\frac{|\\langle \\alpha|H_Z-H_X|0\\rangle|}{E_\\alpha - E_0}\$")
axvline(gaploc, ls = "--", c = "k", alpha = 0.5)

legend(frameon = false)

reg_func = x -> regularize.(x, ε = 1e-4)
# reg_func = x -> regularize.(x, ε=1.0e-1)
# inv_mag = smoothen(reg_func(abs.(magnetizations[top_idx, :])), coarse_times, navg=128)
inv_mag = reg_func(abs.(magnetizations[top_idx, :]))
inv_mag = regularize.(smoothen(smoothen(magnetizations[top_idx, :], coarse_times) .|> abs, coarse_times, navg = 2048))

subplot(212)
scale_factor = 1 .+ abs.(complex_coordinate(top_idx, nx_coarse, ny_coarse, nz_coarse)) .^ 2
plot(coarse_times, (smoothen(all_flucs[top_idx, :], coarse_times) .- mean_flucs))
# plot(coarse_times, (smoothen(scale_factor .^ 2 .* all_flucs[top_idx, :], coarse_times) .- mean_scaled_flucs))
# semilogy(coarse_times, (smoothen(scale_factor.^2 .* all_flucs[top_idxs[i], :], coarse_times) .- mean_scaled_flucs))

# plot(coarse_times, smoothen(scale_factor .^ 2 .* all_flucs[top_idx, :] .* inv_mag, coarse_times))
# semilogy(coarse_times, smoothen(scale_factor.^2 .* all_flucs[top_idxs[i], :] .* inv_mag, coarse_times))

axvline(gaploc, ls = "--", c = "k", alpha = 0.5)
xlim(0, 1)
# ylim(1e-1, 1e2)
ylim(0, )
xlabel("\$s\$")
ylabel("\$ \\langle\\delta \\bar z_i(s) \\delta z_i(s) \\rangle / |m_i(s)|\$")

tight_layout()
display(gcf())
PythonPlot.close();

### Susceptibility

In [ ]:
# figure(figsize = (4, 4))
# ax = subplot(211)
# semilogy(exact_times, abs.(overlap_01_z .- overlap_01_x) ./ ((λ[2, :] .- λ[1, :])), label = "\$\\alpha = 1\$")
# # plot(exact_times, abs.(overlap_02_z .- overlap_02_x) ./ ((λ[3, :] .- λ[1, :])), label="\$\\alpha = 2\$", lw=2)
# # semilogy(exact_times, (abs.(overlap_01_z .- overlap_01_x) ./ ((λ[2, :] .- λ[1, :]))) .+ (abs.(overlap_02_z .- overlap_02_x) ./ ((λ[3, :] .- λ[1, :]))), "-k", lw=2)
# xlim(0.0, 1.0)
# ylim(1e-1, 1e3)
# ax.set_xticklabels([])
# ylabel("\$\\frac{|\\langle \\alpha|H_Z-H_X|0\\rangle|}{E_\\alpha - E_0}\$")
# axvline(gaploc, ls = "--", c = "k", alpha = 0.5)

# legend(frameon = false)

# scaled_flucs(scale_factor, spin_nr) = scale_factor .^ 2 .* smoothen(all_flucs[spin_nr, :], coarse_times, navg = 128)
# suscept_denom(spin_nr) = smoothen(map(x -> x[2] - x[1], zip(magnetizations[spin_nr, 1:end-1], magnetizations[spin_nr, 2:end])), coarse_times[2:end], navg = 128)

# scaled_flucs(scale_factor, spin_nr) = scale_factor .^ 2 .* all_flucs[spin_nr, :]
# suscept_denom(spin_nr) = map(x -> x[2] - x[1], zip(magnetizations[spin_nr, 1:end-1], magnetizations[spin_nr, 2:end]))

# subplot(212)
# scale_factor = 1 .+ abs.(complex_coordinate(top_idx, nx_coarse, ny_coarse, nz_coarse)) .^ 2
# # scale_factor = smoothen(1 .+ abs.(complex_coordinate(top_idxs[i], nx_coarse, ny_coarse, nz_coarse)).^2, coarse_times, navg=128)
# # semilogy(coarse_times[2:end], map(x -> x[2] - x[1], zip(scaled_flucs(scale_factor, top_idxs[i])[1:end-1], scaled_flucs(scale_factor, top_idxs[i])[2:end])) ./ suscept_denom(top_idxs[i]) .|> abs)

# semilogy(coarse_times[2:end],
# 	smoothen(map(x -> x[2] - x[1], zip(scaled_flucs(scale_factor, top_idx)[1:end-1], scaled_flucs(scale_factor, top_idx)[2:end])) ./ suscept_denom(top_idx),
# 		coarse_times[2:end], navg = 512) .|> abs)

# axvline(gaploc, ls = "--", c = "k", alpha = 0.5)
# xlim(0, 1)
# # ylim(1e-1, 1e2)
# xlabel("\$s\$")
# ylabel("\$ \\chi_i(s)\$")

# tight_layout()
# display(gcf())
# PythonPlot.close();

In [ ]:
figure(figsize = (5, 2))


subplot(121)
plot(coarse_times, sum([(all_flucs[top_idxs[k], :] .- mean(all_flucs, dims = 1)[1, :]) for k in 1:1]))
plot(coarse_times, sum([(all_flucs[top_idxs[k], :] .- mean(all_flucs, dims = 1)[1, :]) for k in 1:2]))
axvline(gaploc, c = "r")
# axvline(gap_interval[1], c="k")
# axvline(gap_interval[end], c="k")
xlim(0, 1)
ylim(0)

subplot(122)
plot(coarse_times, all_flucs[top_idxs[1], :] .- mean(all_flucs, dims = 1)[1, :])
plot(coarse_times, all_flucs[top_idxs[2], :] .- mean(all_flucs, dims = 1)[1, :])
plot(coarse_times, all_flucs[top_idxs[4], :] .- mean(all_flucs, dims = 1)[1, :])
axvline(gaploc, c = "r")
# axvline(gap_interval[1], c="k")
# axvline(gap_interval[end], c="k")
xlim(0, 1)
# ylim(0, )
tight_layout()
display(gcf())